In [ ]:
import pandas as pd
import joblib as joblib

In [ ]:
df = pd.read_csv("hour.csv", parse_dates=["dteday"])
df.head()

In [ ]:
df.info()

In [ ]:
#Dropping the columns we don't need from X, and defining the target variables Y
X = df.drop(columns=["instant", "cnt", "casual", "registered"])
y = df["cnt"]

In [ ]:
import numpy as np

In [ ]:
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OrdinalEncoder, FunctionTransformer

In [ ]:
# This function defines a forward fill method to fill NAs
def ffill_missing(ser):
    return ser.fillna(method="ffill")

In [ ]:
#Define the forward fill imputer
ffiller = FunctionTransformer(ffill_missing)

In [ ]:
#Define the weathersit data encoder
weather_enc = make_pipeline(
    ffiller,
    OrdinalEncoder(
        handle_unknown="use_encoded_value", unknown_value=X["weathersit"].nunique()
    ),
)
weather_enc

In [ ]:
ct = make_column_transformer(
    (ffiller, make_column_selector(dtype_include=np.number)),
    (weather_enc, ["weathersit"]),
)
ct

In [ ]:
from sklearn.pipeline import FeatureUnion, make_union

In [ ]:
#This function is used further ahead to consider binary feature of wether the day is weekday or weekend
def is_weekend(data):
    return (
        data["dteday"]
        .dt.day_name()
        .isin(["Saturday", "Sunday"])
        .to_frame()
    )

In [ ]:
#This function is used to tell how many years have elapsed since the beginning of operation (began in 2011)
def year(data):
    # Our reference year is 2011, the beginning of the training dataset
    return (data["dteday"].dt.year - 2011).to_frame()

In [ ]:
preprocessing = FeatureUnion([
    ("is_weekend", FunctionTransformer(is_weekend)),
    ("year", FunctionTransformer(year)),
    ("column_transform", ct)
])
preprocessing

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
reg = Pipeline([("preprocessing", preprocessing), ("model", RandomForestRegressor())])
reg

In [ ]:
X_train, y_train = X.loc[X["dteday"] < "2012-10"], y.loc[X["dteday"] < "2012-10"]
X_test, y_test = X.loc["2012-10" <= X["dteday"]], y.loc["2012-10" <= X["dteday"]]

In [ ]:
reg.fit(X_train, y_train)

In [ ]:
 #Dump the model
    
joblib.dump(reg, 'bikes.joblib')

In [ ]:
reg.score(X_test, y_test)

In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))

df.loc[df["dteday"] < "2012-10"].set_index("instant")["cnt"].plot(ax=ax, label="Train")
df.loc["2012-10" <= df["dteday"]].set_index("instant")["cnt"].plot(ax=ax, label="Test")

pd.Series(y_pred, index=df.loc["2012-10" <= df["dteday"], "instant"]).plot(ax=ax, color="k", label="Prediction")

ax.legend(loc=2, shadow=True, facecolor="0.97")
ax.set_xlim(15100, 15500)

---

In [ ]:
X_train.head()

In [ ]:
reg.predict(pd.DataFrame([[
    pd.to_datetime("2012-11-01"),
    10,
    "Clear, Few clouds, Partly cloudy, Partly cloudy",
    0.3,
    0.31,
    0.8,
    0.0,
]], columns=[
    'dteday',
    'hr',
    'weathersit',
    'temp',
    'atemp',
    'hum',
    'windspeed'
]))